In [1]:
import numpy as np
import pandas as pd
import os
import argparse
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import tweepy
import matplotlib.pyplot as plt

ImportError: cannot import name 'enums' from 'google.cloud.language_v1' (C:\Users\hongx\AppData\Local\Programs\Python\Python38\lib\site-packages\google\cloud\language_v1\__init__.py)

In [4]:
access_key=""
access_secret=""
consumer_key=""
consumer_secret=""

auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=""

In [10]:
# screen name of the account to be fetched
screen_name = "JessicaXinH"
  
# fetching the statuses
statuses = api.user_timeline(screen_name)

#export results to a txt file
print(screen_name + ": " + str(len(statuses)) + " number of statuses have been fetched.")
lines = [screen_name + 'Timeline', str(statuses)]
with open(screen_name+'_Timeline.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
print(screen_name+'_Timeline.txt')

print("Print 20 most recent statuses")
count = 0
for status in statuses:
    count = count + 1
    print(str(count) + status.text, end = "\n")

JessicaXinH: 18 number of statuses have been fetched.
JessicaXinH_Timeline.txt
Print 20 most recent statuses


In [ ]:
file = open(screen_name+'_Timeline.txt',"r+")
text= file.readline()
#text = "I am very happy"

In [82]:
posts = api.user_timeline(screen_name = screen_name, count = 20, lang = "en", tweet_mode = "extended", include_rts = True, exclude_replies = True )
df = pd.DataFrame([tweet.full_text for tweet in posts], columns = ['Tweets'])
client = language_v1.LanguageServiceClient()
type_ = enums.Document.Type.PLAIN_TEXT
language = "en"
document = {"content": text, "type": type_, "language": language}

encoding_tpye = enums.EncodingType.UTF8

response = client.analyze_sentiment(document, encoding_type=encoding_type)

In [ ]:
#Get sentiment of sentence
for sentence in response.sentences:
    print(u"Sentence text: {}".format(sentence.text.content))
    print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
    print(u"Sentence sentiment magnitude: {}".format(sentence.sentiment.magnitude))
    df.at[sentence,'Sentence_score'] = response.document_sentiment.score
    df.at[sentence,'Sentence_magnitude'] = response.document_sentiment.magnitude
#get sentiment of the document
print(u"Document sentiment score: {}".format(response.document_sentiment.score))
print(u"magnitude: {}".format(response.document_sentiment.magnitude))

In [ ]:
plt.figure(figsize=(6,6))
for i in range(0,df.shape[0]):
    plt.scatter(df.at[i, 'Sentence_score'], df.at[i, 'Sentence_magnitude'])
plt.title(screen_name+'_Sentiment_Analysis')
plt.xlabel('Sentence_score')
plt.ylabel('Sentence_magnitude')
plt.show()
plt.close()